In [48]:
import pprint as pp
import requests 
import json
from datetime import date
import csv
import datetime
import pandas as pd
import time
from time import mktime
from datetime import datetime, timedelta
import numpy as np

In [49]:
#def getPushshiftData(query, after, before, sub):
def getPushshiftData(after, before, sub):
    #url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    url = 'https://api.pushshift.io/reddit/search/submission/?size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    #url = 'https://api.pushshift.io/reddit/search/submission/?link_flair_text=Resources&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    #SEE ABOUT SOME OTHER THING FOR FLAIR doesn't seem to be working
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [50]:
def collectSubData(subm):
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']
    
    subData.append((sub_id,title,url,author,score,created,numComms,permalink,flair))
    subStats[sub_id] = subData

In [51]:
#Subreddit to query
#sub='PS4'
sub='COVID19_support'
#before and after dates
#before = "1538352000" #October 1st
#after = "1514764800"  #January 1st 
before = "1585699200"
after = "1580515200"
#query = "Screenshot"
subCount = 0
subStats = {}

In [52]:
#data = getPushshiftData(query, after, before, sub)# Will run until all posts have been gathered 
data = getPushshiftData(after, before, sub)
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftData(after, before, sub)
    #data = getPushshiftData(query, after, before, sub)
    
print(len(data))

https://api.pushshift.io/reddit/search/submission/?link_flair_text=Resources&size=1000&after=1580515200&before=1585699200&subreddit=COVID19_support
1000
2020-03-22 06:04:29
https://api.pushshift.io/reddit/search/submission/?link_flair_text=Resources&size=1000&after=1584871469&before=1585699200&subreddit=COVID19_support
713
2020-03-31 19:46:24
https://api.pushshift.io/reddit/search/submission/?link_flair_text=Resources&size=1000&after=1585698384&before=1585699200&subreddit=COVID19_support
0


In [53]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

1713 submissions have added to list
1st entry is:
Welcome to COVID19 Support! created: 2020-02-14 21:50:24
Last entry is:
(Australia) Psychologists can offer online sessions - some are covered by gov mental health plan scheme created: 2020-03-31 19:46:24


In [54]:
def updateSubs_file():
    upload_count = 0
    #location = "\\Reddit Data\\"
    
    #print("input filename of submission file, please add .csv")
    filename = "redditdata.csv"#input()
    #file = location + filename
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
    # with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded") #updateSubs_file()

In [55]:
updateSubs_file()

1713 submissions have been uploaded
